In [12]:
from oai_tool import tool
from typing import Annotated
from pydantic import Field
from enum import Enum
import json
import openai
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
client = openai.OpenAI(api_key=OPENAI_API_KEY)
GPT_MODEL = "gpt-4o"

def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [13]:
class TemperatureUnit(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"
        

@tool
def get_current_weather(
    location: Annotated[str, Field(description="The city and state, e.g. San Francisco, CA")],
    format: Annotated[TemperatureUnit, Field(description="The temperature unit to use. Infer this from the user's location.")]
) -> dict:
    """Get the current weather"""
    return json.dumps({
        "location": location,
        "temperature": "25°C" if format == TemperatureUnit.CELSIUS else "77°F",
        "condition": "Sunny"
    })

In [14]:
print(json.dumps(get_current_weather._schema, indent=4))

{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "format": {
                    "type": "string",
                    "enum": [
                        "celsius",
                        "fahrenheit"
                    ],
                    "description": "The temperature unit to use. Infer this from the user's location."
                }
            },
            "required": [
                "location",
                "format"
            ]
        }
    }
}


In [15]:
tools = [get_current_weather]
tool_dispatch = {tool._schema['function']['name']: tool for tool in tools}
messages = [
        {"role": "system", "content": "You fetch the current weather for a user."},
        {"role": "user", "content": "What's the weather in New York?"}
    ]
response = chat_completion_request(
    messages=messages,
    tools=[tool._schema for tool in tools],
    tool_choice="auto",
    model=GPT_MODEL
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [16]:
tool_message = response.choices[0].message
messages.append(tool_message)
tool_call = response.choices[0].message.tool_calls[0]
tool_call

ChatCompletionMessageToolCall(id='call_ivhG4WEQQGakhid25aJHWUxB', function=Function(arguments='{"location":"New York, NY","format":"fahrenheit"}', name='get_current_weather'), type='function')

In [17]:
tool_calls = response.choices[0].message.tool_calls
if tool_calls:
    # There could be multiple tool calls, but for this example we only expect one
    tool_call = tool_calls[0]
    tool_call_id = tool_call.id
    tool_name = tool_call.function.name
    tool = tool_dispatch[tool_name]
    # This is where the future validation logic code would go
    # For now, we just assume the tool call is valid
    tool_args = json.loads(tool_call.function.arguments)
    tool_response = tool(**tool_args)
    tool_message = {
        "role": "tool",
        "content": tool_response,
        "tool_call_id": tool_call_id,
        "name": tool_name,
    }
    
 

# raw_json_response = response.choices[0].message.tool_calls[0].function.arguments

In [18]:

tool_message, tool_response

({'role': 'tool',
  'content': '{"location": "New York, NY", "temperature": "77\\u00b0F", "condition": "Sunny"}',
  'tool_call_id': 'call_ivhG4WEQQGakhid25aJHWUxB',
  'name': 'get_current_weather'},
 '{"location": "New York, NY", "temperature": "77\\u00b0F", "condition": "Sunny"}')

In [19]:
messages_to_send = messages + [tool_message]
messages_to_send

[{'role': 'system',
  'content': 'You fetch the current weather for a user and write a story about it.'},
 {'role': 'user', 'content': "What's the weather in New York?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ivhG4WEQQGakhid25aJHWUxB', function=Function(arguments='{"location":"New York, NY","format":"fahrenheit"}', name='get_current_weather'), type='function')]),
 {'role': 'tool',
  'content': '{"location": "New York, NY", "temperature": "77\\u00b0F", "condition": "Sunny"}',
  'tool_call_id': 'call_ivhG4WEQQGakhid25aJHWUxB',
  'name': 'get_current_weather'}]

In [20]:
model_response_with_function_call = client.chat.completions.create(
    model="gpt-4o",
    messages=messages + [tool_message],
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [21]:
model_response_with_function_call.choices[0].message.content

"Once upon a time, in the bustling heart of New York City, it was a perfect day. The sun shone brightly, casting a golden glow over the iconic skyline and making everything it touched seem to sparkle with life. It was 77°F—an ideal temperature for both locals and tourists to enjoy the myriad of activities the city had to offer.\n\nIn Central Park, families and friends gathered, spreading checkered blankets and sharing picnic feasts, their laughter echoing among the lush greenery and tranquil ponds. Joggers and cyclists cruised along the park's winding paths, reveling in the warm rays that filtered through the trees.\n\nDown in Times Square, the electronic billboards glistened under the radiant sun, creating a dazzling spectacle. Street performers entertained passersby, their energy seemingly fueled by the bright, cloudless sky. In the surrounding avenues, the city's famed yellow cabs zipped by, ferrying passengers to their various urban adventures.\n\nHeading towards the Hudson River, 